# Creating and Assigning an Executor

If multiple electrons can use the same executor, create one executor and assign it to the electrons rather than creating an executor every time an electron is instantiated.

## Context

Often, multiple electrons can use the same executor. Examples include:
* Multiple instances of the same electron class
* Electrons that are structurally or functionally similar (sibling subclasses, for example)
* Electrons that use the same platform because they perform similar types of computation

Especially if the host or system behind the executor parallelizes its assigned jobs, then creating a different executor for each electron consumes resources unnecessarily.

As well, instantiating multiple executors complicates code maintenance. Defining the executor once means any modification can be done once only, in one place.

## Best Practice

When creating  multiple electrons that use the same executor resource, instantiate the executor and then assign the executor to electrons as they are instantiated. You can think of this technique as similar to (but not exactly!) the popular Singleton coding pattern. 

## Example

Contrast the two examples below. 

### Not Recommended

This example demonstrates a questionable approach: creating an executor for every new instance of an electron.

In [62]:
import covalent as ct
import random
from covalent.executor import LocalExecutor

# Technique 1 (not so good):

def dummy():
    pass

@ct.electron(executor=LocalExecutor())
def task_1():
    return random.sample(range(10, 30), 5)

@ct.electron(executor=LocalExecutor())
def task_2(x_list):

    squares = []
    for x in x_list:
        squares.append(x ** 2)
    
    return squares

@ct.lattice
def workflow_1():
    random_list = task_1()
    return task_2(random_list)

id = ct.dispatch(workflow_1)()
res = ct.get_result(id, wait=True)
print(res)


Lattice Result
status: COMPLETED
result: [576, 121, 100, 256, 529]
input args: []
input kwargs: {}
error: None

start_time: 2023-02-03 21:40:15.847556
end_time: 2023-02-03 21:40:15.973169

results_dir: /Users/mini-me/agnostiq/covalent/doc/source/developer/patterns/results
dispatch_id: b03ef78b-acbd-4bf9-a643-af1194303f5b

Node Outputs
------------
task_1(0): [24, 11, 10, 16, 23]
task_2(1): [576, 121, 100, 256, 529]



### Improved

This example demonstrates the better technique: creating one executor and assigning it to each new instance of the electron.

In [63]:
import covalent as ct
import random
from covalent.executor import DaskExecutor

# Technique 2 (better):

custom_executor = DaskExecutor()

@ct.electron(executor=custom_executor)
def task_1():
    return random.sample(range(10, 30), 5)


@ct.electron(executor=custom_executor)
def task_2(x_list):

    squares = []
    for x in x_list:
        squares.append(x ** 2)
    
    return squares

@ct.lattice
def workflow_2():
    random_list = task_1()
    return task_2(random_list)

id = ct.dispatch(workflow_1)()
res = ct.get_result(id, wait=True)
print(res)


Lattice Result
status: COMPLETED
result: [100, 400, 841, 289, 324]
input args: []
input kwargs: {}
error: None

start_time: 2023-02-03 21:40:24.365009
end_time: 2023-02-03 21:40:24.438503

results_dir: /Users/mini-me/agnostiq/covalent/doc/source/developer/patterns/results
dispatch_id: 767b6351-b747-4bc5-83c4-5a5a8689ec99

Node Outputs
------------
task_1(0): [10, 20, 29, 17, 18]
task_2(1): [100, 400, 841, 289, 324]



## Keywords

- Return values
- Results

## Benefits

- Efficiency
- Conservation of Resources
- Maintainability

## See Also